In [ ]:
#!pip install tweepy
#!pip install ibm_watson
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import tweepy as tw
from textblob import TextBlob

In [ ]:
filename='fake_tweets_id.jsonl'
df =pd.read_json(open(filename,"r", encoding='utf8'),lines=True,orient= 'records')

In [ ]:
user_id_list = [obj['id'] for obj in df['user']]  
df_user_id_list = pd.DataFrame(user_id_list).drop_duplicates().reset_index(drop=True)[0].values.tolist()
df_user_id_list

In [ ]:
# import the module 
import tweepy 
import json

# assign the values accordingly 
consumer_key = "put consumer_key "
consumer_secret = "consumer_secret"
access_key = "access_key"
access_secret = "access_secret"


# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 

# set access to user's access key and access secret 
auth.set_access_token(access_key, access_secret) 

# calling the api 
api = tweepy.API(auth) 


# generating the tweets

def tweets_of_user_ids(user_list_id):
    user_tweets = {}
    notvalid=[]
    for id_num in user_list_id:
        user_id = id_num
        try:
            user_tweets[id_num] = api.user_timeline(user_id)
        except:
            #print "NotOk :",row.id
            notvalid.append(user_id)
            #print x, "added to valid"
        

    users_dict_tweets = {}
    for user_id, statuse in user_tweets.items():
        users_dict_tweets[user_id] = []
        for tweet in statuse:
           users_dict_tweets[user_id].append(tweet.text) 
    print(notvalid)
    return users_dict_tweets

tweets = tweets_of_user_ids(df_user_id_list)

In [ ]:
# data cleaning 
def str_clean(tweets_list):
    tweets = []
    for text in tweets_list:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        text = re.sub(r'rt' , '', text)
        # tokenizing the text
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
        # words = [w for w in words if not w in stop_words]
        #words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        #print(words)
        tweets.append(words)
    return tweets
            
# Extracting tweets from the users dictionary
def extract_tweet(dic):
    clean_tweets={}
    for user_id, tweets_list in dic.items():
        clean_tweets[user_id]= str_clean(tweets_list)
    return clean_tweets

# Word counter
def word_counter(dic):
    tokenized_tweets = {}
    for user_id, tweets_list in dic.items():
        lis = []
        for tweet in tweets_list:
            lis.extend(word_tokenize(tweet))
        word_count = FreqDist(lis)
        tokenized_tweets[user_id] = word_count
    return tokenized_tweets

# Word counter without the stopwords 
def word_counter_stop_w(dic):
    st_w = stopwords.words('english')
    tokenized_tweets = {}
    for user_id, tweets_list in dic.items():
        lis = []
        for tweet in tweets_list:
            lis.extend(word_tokenize(tweet))
        # remove stopwords
        lis_stop_w = [word for word in lis if word not in st_w]
        word_count = FreqDist(lis_stop_w)
        tokenized_tweets[user_id] = word_count
    return tokenized_tweets

# Post_tag the tweets
def pos_tag_tweets(dic):
    pos_tag_dic = {}
    chunk_grammar = "NP: {<JJ><NN>}"
    chunk_parser = RegexpParser(chunk_grammar) 
    for user_id, tweets in dic.items():
        tweet_list = []
        for tweet in tweets:
            tweet_tokenized = word_tokenize(tweet)
            #print(tweet_tokenized)
            tweet_pos_tag = pos_tag(tweet_tokenized)
            #print(tweet_pos_tag)
            [('Now', 'RB'), ('im', 'VBZ'), ('up', 'RP'), ('watch', 'JJ'), ('60', 'CD'), ('days', 'NNS'), ('in', 'IN')]
            tweet_parser = [tag[0] for tag in tweet_pos_tag if tag[1]=='JJ']
            #print(tweet_parser)
            #tweet_list.append(chunk_parser.parse(tweet_parser))
            tweet_list.append(tweet_parser)
        pos_tag_dic[user_id] = tweet_list
    return pos_tag_dic

extr_tweets = extract_tweet(tweets) 

In [ ]:
def tweets_to_string(dicto):
    dic={}
    for user_id, string_list in dicto.items():
        dic[user_id]= ' '.join(string_list)
    return dic
extr_tweets_strings = tweets_to_string(extr_tweets)

In [ ]:
df=pd.DataFrame.from_dict(extr_tweets_strings, orient='index', columns=['tweet'])


In [176]:
df['Polarity']=df['tweet'].apply(lambda x: TextBlob(x).sentiment)
df['Subjectivity']=df['tweet'].apply(lambda x: TextBlob(x).sentiment)
df

,tweet,Polarity,Subjectivity
429437147,and you ve got soo much more to achieve this i...,"(0.12499999999999997, 0.4091666666666667)","(0.12499999999999997, 0.4091666666666667)"
507252116,happy bihday superstar rajinikanth hbdsupersta...,"(0.43783068783068785, 0.7134920634920634)","(0.43783068783068785, 0.7134920634920634)"
1065961267440033794,good what are you grateful for today existing ...,"(0.3385974025974026, 0.5524675324675324)","(0.3385974025974026, 0.5524675324675324)"
750251127673729024,every man and woman on this eah is born to lea...,"(0.2408936588103255, 0.5157435465768798)","(0.2408936588103255, 0.5157435465768798)"
482175176,govmurphy woohoo omg i love this so much egolb...,"(0.1299107142857143, 0.4399496336996337)","(0.1299107142857143, 0.4399496336996337)"
...,...,...,...
1120598108482736128,saluting the warrior spirit of cancer esconpro...,"(0.17820634920634923, 0.4993174603174603)","(0.17820634920634923, 0.4993174603174603)"
15611093,adamsinger until you live there and realize th...,"(0.11846193022663612, 0.5541889483065954)","(0.11846193022663612, 0.5541889483065954)"
402622605,ଏକ ବୟସ ସମୟ ଏକ ବୟସ ସମୟ ଏକ ଆଇନ କ ଣ କଣ ଏକ ଆଇନ କ ଣ...,"(0.8, 0.75)","(0.8, 0.75)"
1243862687198654466,rudogumbo got you sis pmarkhansen yktv rudogum...,"(0.39112554112554115, 0.47886904761904764)","(0.39112554112554115, 0.47886904761904764)"
